In [ ]:
import numpy as np
import os
import pandas as pd
import sys

from matplotlib import pyplot as plt
from tqdm import tqdm

project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)
import config

from experiments.MNIST import ExperimentADeLEn, ExperimentSupervised, ExperimentSVM

# Pollution experiments using MNIST


In [ ]:
pollution_exp = [0, .05, .1, .2]

## Table

In [ ]:
from itertools import chain
def generate_multi_df(data:list, parent_index:list, child_index:list) -> pd.DataFrame:
    index_names = [
        [x for x in parent_index for _ in child_index],
        child_index*len(parent_index)
    ]

    tuples = list(zip(*index_names))
    index = pd.MultiIndex.from_tuples(tuples)
    return pd.DataFrame(chain.from_iterable(data), index=index)

In [ ]:
exp = ExperimentADeLEn(.1, .2, 2)
result_dir = exp.config()['save_result_dir']
df = pd.read_csv(os.path.join(result_dir, 'metrics.csv'), index_col=0)
acc = df['Accuracy'].mean(), df['Accuracy'].std()
prec = df['Precision'].mean(), df['Precision'].std()
rec = df['Recall'].mean(), df['Recall'].std()
f1 = df['F1'].mean(), df['F1'].std()
auc = df['AUC'].mean(), df['AUC'].std()


data = [acc, prec, rec, f1, auc]
parent_index = ['Acc', 'Prec', 'Recall', 'F1', 'AUC']
child_index = ['Mean', 'Std']
df = generate_multi_df(data, parent_index, child_index).T
df

In [ ]:
exp = ExperimentADeLEn(.1, .2, 2)
result_dir = exp.config()['save_result_dir']
df = pd.read_csv(os.path.join(result_dir, 'metrics.csv'), index_col=0)
df['Accuracy'].mean(), df['Accuracy'].std(),

In [ ]:
(df['Accuracy'].mean(), df['Accuracy'].std()), (df['Precision'].mean(), df['Precision'].std()), (df['Recall'].mean(), df['Recall'].std()), (df['F1'].mean(), df['F1'].std()), (df['AUC'].mean(), df['AUC'].std())

In [ ]:
exp = ExperimentSupervised(.1, .2)
result_dir = exp.config()['save_result_dir']
df = pd.read_csv(os.path.join(result_dir, 'metrics.csv'), index_col=0)

In [ ]:
(df['Accuracy'].mean(), df['Accuracy'].std()), (df['Precision'].mean(), df['Precision'].std()), (df['Recall'].mean(), df['Recall'].std()), (df['F1'].mean(), df['F1'].std()), (df['AUC'].mean(), df['AUC'].std())

## Images

In [ ]:
def generate_score_histogram(score_df:pd.DataFrame, normal_bins=10, anomaly_bins=10):
    from matplotlib.ticker import MaxNLocator

    with plt.style.context('seaborn-colorblind'):
        fig = plt.figure(figsize=(6, 4))
        ax = plt.gca()
        ax.hist(score_df['Normal'], bins=normal_bins, alpha=.7, label='Normal')
        ax.hist(score_df['Anomaly'], bins=anomaly_bins, alpha=.7, label='Anomaly')
        ax.legend(fontsize='x-large')
        ax.xaxis.set_major_locator(MaxNLocator(nbins=5)) 
        ax.yaxis.set_major_locator(MaxNLocator(nbins=5))
        ax.set_ylabel('Samples', fontsize='x-large')
        ax.set_xlabel('Score', fontsize='x-large')
        ax.tick_params(axis='both', which='major', labelsize='large')
        ax.grid()
        
        return fig

### ADeLEn

In [ ]:
for d in [2, 5, 10]:
    for pollution in pollution_exp:
        exp = ExperimentADeLEn(.1, pollution, d)
        result_dir = exp.config()['save_result_dir']
        img_dir = exp.config()['save_imgs_dir']

        df = pd.read_csv(os.path.join(result_dir, 'metrics.csv'), index_col=0)
        idx = df['AUC'].apply(lambda x: abs(x - df['AUC'].mean())).idxmin()

        score_df = pd.read_pickle(os.path.join(result_dir, 'sample_score.pkl'))[f'It {idx+1}']
        fig = generate_score_histogram(score_df)
        fig.savefig(os.path.join(img_dir, f'score_histogram_{pollution}.pdf'), bbox_inches='tight')
        plt.close(fig)


### Supervised

In [ ]:
for pollution in pollution_exp:
    exp = ExperimentSupervised(.1, pollution)
    result_dir = exp.config()['save_result_dir']
    img_dir = exp.config()['save_imgs_dir']

    df = pd.read_csv(os.path.join(result_dir, 'metrics.csv'), index_col=0)
    idx = df['AUC'].apply(lambda x: abs(x - df['AUC'].mean())).idxmin()

    score_df = pd.read_pickle(os.path.join(result_dir, 'sample_score.pkl'))[f'It {idx+1}']
    score_df['Normal'] = score_df['Normal'].apply(lambda x: x.item() if x is not None else x)
    score_df['Anomaly'] = score_df['Anomaly'].apply(lambda x: x.item() if x is not None else x)
    
    fig = generate_score_histogram(score_df, normal_bins=(1 if pollution==.2 else 10))
    fig.savefig(os.path.join(img_dir, f'score_histogram_{pollution}.pdf'), bbox_inches='tight')
    plt.close(fig)

# SVM

In [ ]:
for pollution in pollution_exp:
    exp = ExperimentSVM(.1, pollution)
    result_dir = exp.config()['save_result_dir']
    img_dir = exp.config()['save_imgs_dir']

    df = pd.read_csv(os.path.join(result_dir, 'metrics.csv'), index_col=0)
    idx = df['AUC'].apply(lambda x: abs(x - df['AUC'].mean())).idxmin()

    score_df = pd.read_pickle(os.path.join(result_dir, 'sample_score.pkl'))[f'It {idx+1}']
    
    fig = generate_score_histogram(score_df)
    fig.savefig(os.path.join(img_dir, f'score_histogram_{pollution}.pdf'), bbox_inches='tight')
    plt.close(fig)